<a href="https://colab.research.google.com/github/draculess99/Tensorflow2/blob/master/%20Section6_transfer_learning_and_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

In [2]:
!pip install tqdm

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-07-05 17:41:14--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   126MB/s    in 0.5s    

2019-07-05 17:41:15 (126 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-alpha0'

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [0]:
IMG_SHAPE = (128, 128, 3)

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [19]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
base_model.trainable = True

In [21]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [23]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [29]:
 model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [33]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [34]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [35]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 23s 1s/step - loss: 0.2405 - accuracy: 0.9010 - val_loss: 0.0995 - val_accuracy: 0.9580
Epoch 2/5
16/16 [==============================] - 7s 418ms/step - loss: 0.0376 - accuracy: 0.9945 - val_loss: 0.1000 - val_accuracy: 0.9580
Epoch 3/5
16/16 [==============================] - 7s 419ms/step - loss: 0.0122 - accuracy: 0.9995 - val_loss: 0.1086 - val_accuracy: 0.9620
Epoch 4/5
16/16 [==============================] - 7s 425ms/step - loss: 0.0067 - accuracy: 0.9995 - val_loss: 0.1436 - val_accuracy: 0.9590
Epoch 5/5
16/16 [==============================] - 7s 423ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.1480 - val_accuracy: 0.9600


In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [37]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9599999785423279


In [0]:
 # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 # fine tuning 
# unfreeze top layer

In [0]:
base_model.trainable = True

In [42]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 155


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [48]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 20s 1s/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.3639 - val_accuracy: 0.9380
Epoch 2/5
16/16 [==============================] - 7s 421ms/step - loss: 8.2263e-04 - accuracy: 1.0000 - val_loss: 0.3244 - val_accuracy: 0.9480
Epoch 3/5
16/16 [==============================] - 7s 430ms/step - loss: 2.6798e-04 - accuracy: 1.0000 - val_loss: 0.4789 - val_accuracy: 0.9240
Epoch 4/5
16/16 [==============================] - 7s 430ms/step - loss: 2.5398e-04 - accuracy: 1.0000 - val_loss: 0.3958 - val_accuracy: 0.9410
Epoch 5/5
16/16 [==============================] - 7s 437ms/step - loss: 9.6458e-05 - accuracy: 1.0000 - val_loss: 0.3718 - val_accuracy: 0.9460


In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [50]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9459999799728394
